# SSIM comparison
- Baseline에서 ssim이 0.9보다 낮은 경우, RL에서는 ssim이 얼마나 나올지?

In [116]:
import pandas as pd
import os

# Step 1: 비교할 CSV 파일 목록 생성
methods = ["kt", "lg", "combine"]
gops = [1, 10, 30]
thresholds = [30, 35, 40]

base_path = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60"

csv_files = [
    f"{base_path}/raw_data_{method}_gop{gop}_threshold_{threshold}.csv"
    for method in methods
    for gop in gops
    for threshold in thresholds
]

# Step 2: 추가적인 threshold_data2 파일 로드 및 필터링
# data2 파일 로드
threshold_data2 = {
    30: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_30.csv'),
    35: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_35.csv'),
    40: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_40.csv')
}

filtered_data2 = {
    key: df[df['SSIM'] != 0] for key, df in threshold_data2.items()
}

# Step 3: 각 CSV 파일 처리
summary_data = []

for file_path in csv_files:
    try:
        # CSV 파일 로드
        data = pd.read_csv(file_path)
        
        # SSIM != 0 인 데이터 필터링
        filtered_data = data[data['ssim'] != 0]
        
        # 평균 SSIM 계산
        mean_ssim = filtered_data['ssim'].mean()
        
        # 파일명 추출
        file_name = os.path.basename(file_path)
        
        # 결과 저장
        summary_data.append({
            'file': file_name,
            'mean_ssim': mean_ssim
        })
        
        print(f"Processed: {file_name}, Mean SSIM: {mean_ssim:.6f}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Step 4: threshold_data2 파일 처리
for threshold, df in filtered_data2.items():
    try:
        mean_ssim = df['SSIM'].mean()
        summary_data.append({
            'file': f'step_logs_1_1_280_{threshold}.csv',
            'mean_ssim': mean_ssim
        })
        print(f"Processed: step_logs_1_1_280_{threshold}.csv, Mean SSIM: {mean_ssim:.6f}")
    except Exception as e:
        print(f"Error processing step_logs_1_1_280_{threshold}.csv: {e}")

# Step 5: 결과 저장
output_folder = os.path.join(base_path, 'comparison')
os.makedirs(output_folder, exist_ok=True)
summary_path = os.path.join(output_folder, 'summary_ssim2.csv')

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(summary_path, index=False)

print("Summary saved at:", summary_path)
print("All files processed successfully.")


Processed: raw_data_kt_gop1_threshold_30.csv, Mean SSIM: 0.960023
Processed: raw_data_kt_gop1_threshold_35.csv, Mean SSIM: 0.959685
Processed: raw_data_kt_gop1_threshold_40.csv, Mean SSIM: 0.959451
Processed: raw_data_kt_gop10_threshold_30.csv, Mean SSIM: 0.890087
Processed: raw_data_kt_gop10_threshold_35.csv, Mean SSIM: 0.934254
Processed: raw_data_kt_gop10_threshold_40.csv, Mean SSIM: 0.955184
Processed: raw_data_kt_gop30_threshold_30.csv, Mean SSIM: 0.829069
Processed: raw_data_kt_gop30_threshold_35.csv, Mean SSIM: 0.895625
Processed: raw_data_kt_gop30_threshold_40.csv, Mean SSIM: 0.937112
Processed: raw_data_lg_gop1_threshold_30.csv, Mean SSIM: 0.959616
Processed: raw_data_lg_gop1_threshold_35.csv, Mean SSIM: 0.959407
Processed: raw_data_lg_gop1_threshold_40.csv, Mean SSIM: 0.959315
Processed: raw_data_lg_gop10_threshold_30.csv, Mean SSIM: 0.921244
Processed: raw_data_lg_gop10_threshold_35.csv, Mean SSIM: 0.950429
Processed: raw_data_lg_gop10_threshold_40.csv, Mean SSIM: 0.959920
P

In [117]:
import pandas as pd
import os


# Step 1: 비교할 CSV 파일 목록 생성
methods = ["kt", "lg", "combine"]
gops = [1, 10, 30]
thresholds = [30, 35, 40]

base_path = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60"

csv_files = [
    f"{base_path}/raw_data_{method}_gop{gop}_threshold_{threshold}.csv"
    for method in methods
    for gop in gops
    for threshold in thresholds
]

# data2 파일 로드
threshold_data2 = {
    30: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_30.csv'),
    35: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_35.csv'),
    40: pd.read_csv('/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1_280_40.csv')
}

# Step 2: data2에서 SSIM != 0인 데이터 필터링
filtered_data2 = {
    key: df[df['SSIM'] != 0] for key, df in threshold_data2.items()
}

# Step 3: 각 CSV 파일 처리
summary_data = []

for file_path in csv_files:
    # 파일명에서 threshold 추출
    threshold = int(file_path.split('_')[-1].split('.')[0].replace('threshold_', ''))

    # data2 파일 선택
    data2 = filtered_data2[threshold]

    # data1 로드 및 필터링
    data1 = pd.read_csv(file_path)
    filtered_data1 = data1[data1['ssim'] != 0]

    # 데이터 병합
    merged_data = filtered_data1.merge(
        data2, left_on='sequence_number', right_on='Frame Number', how='inner'
    )

    # SSIM Advantage 계산
    merged_data['SSIM Advantage (%)'] = ((merged_data['SSIM'] / merged_data['ssim']))

    # 필요한 컬럼만 선택
    logging_data = merged_data[['sequence_number', 'ssim', 'SSIM', 'SSIM Advantage (%)']]

    # 결과 저장 경로 생성
    output_folder = os.path.join(base_path, 'comparison')
    file_path = f"comparison_{os.path.basename(file_path)}"
    output_file = os.path.join(output_folder, file_path)

    # 결과 CSV 파일 저장
    logging_data.to_csv(output_file, index=False)

    print(f"Processed and saved: {output_file}")

    # SSIM 및 SSIM Advantage 평균 계산 및 저장
    mean_ssim = logging_data['ssim'].mean()
    mean_SSIM = logging_data['SSIM'].mean()
    mean_ssim_advantage = logging_data['SSIM Advantage (%)'].mean()
    summary_data.append({
        'file': file_path,
        'threshold': threshold,
        'mean_ssim': mean_ssim,
        'mean_SSIM': mean_SSIM,
        'mean_ssim_advantage': mean_ssim_advantage
    })

# Step 4: 모든 결과의 평균 저장

summary_path = os.path.join(output_folder, 'summary_ssim_advantage.csv')
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(summary_path, index=False)
print("Summary saved")

print("All files processed successfully.")


Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop1_threshold_30.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop1_threshold_35.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop1_threshold_40.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop10_threshold_30.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop10_threshold_35.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop10_threshold_40.csv
Processed and saved: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/comparison/comparison_raw_data_kt_gop30_threshold_30.csv
Processed

In [110]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 분석 대상 parameter
methods = ["kt", "lg", "combine"]
gops = [1, 10, 30]
thresholds = [30, 35, 40]

# CSV 파일이 있는 기본 경로
BASE_PATH = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60"

# 결과 저장 폴더 세팅
freeze_duration_folder = os.path.join(BASE_PATH, "freeze_duration")
graph_folder = os.path.join(freeze_duration_folder, "graph")

# 폴더 없으면 생성
os.makedirs(freeze_duration_folder, exist_ok=True)
os.makedirs(graph_folder, exist_ok=True)

def count_consecutive_zeros(ssim_values):
    """
    ssim_values(list or Series)에서 값이 0인 구간이 연속적으로 몇 개씩 나타나는지 카운트하여
    {연속개수: 빈도} 딕셔너리를 반환
    """
    consecutive_counts = {}
    current_count = 0
    
    for val in ssim_values:
        if val == 0:
            current_count += 1
        else:
            # 0이 아닌 값이 나타나면, 지금까지 센 연속 0의 길이를 기록
            if current_count > 0:
                consecutive_counts[current_count] = consecutive_counts.get(current_count, 0) + 1
                current_count = 0
    # 마지막 값이 0으로 끝났다면 처리
    if current_count > 0:
        consecutive_counts[current_count] = consecutive_counts.get(current_count, 0) + 1
    
    return consecutive_counts

# 전체 파일에 대해 반복 수행
for method in methods:
    for gop in gops:
        for threshold in thresholds:
            filename = f"raw_data_{method}_gop{gop}_threshold_{threshold}.csv"
            filepath = os.path.join(BASE_PATH, filename)
            
            if not os.path.exists(filepath):
                # 파일이 없으면 스킵하거나, 필요한 경우 예외처리
                print(f"[WARNING] File not found: {filepath}")
                continue
            
            # CSV 로드
            df = pd.read_csv(filepath)
            
            # ssim 컬럼에서 연속 0 횟수 계산
            ssim_values = df['ssim'].fillna(0)  # 혹시 모를 NaN 방지
            zero_counts = count_consecutive_zeros(ssim_values)
            
            # 결과 DataFrame 생성 (컬럼명을 "FreezeDuration"으로 변경)
            result_df = pd.DataFrame({
                'FreezeDuration': list(zero_counts.keys()),
                'Count': list(zero_counts.values())
            }).sort_values(by='FreezeDuration', ascending=True)
            
            # 결과 CSV 저장
            result_csv_name = f"freeze_count_{method}_gop{gop}_threshold_{threshold}.csv"
            result_csv_path = os.path.join(freeze_duration_folder, result_csv_name)
            result_df.to_csv(result_csv_path, index=False)
            
            # 그래프 (실선 그래프) 그리기
            plt.figure(figsize=(6, 4))
            plt.plot(result_df['FreezeDuration'], 
                     result_df['Count'], 
                     marker='o', 
                     linestyle='-', 
                     color='blue')
            
            # 제목 및 축 라벨: freeze duration 표현 사용
            plt.title(f"Freeze Duration Distribution\n(method={method}, gop={gop}, threshold={threshold})")
            plt.xlabel("Freeze Duration (Consecutive 0 count)")
            plt.ylabel("Frequency")
            plt.xticks(result_df['FreezeDuration'])  # 연속 freeze 횟수에 맞춰 x축 표시
            plt.grid(True)  # 격자 표시(선택)
            
            # PDF 저장
            graph_pdf_name = f"freeze_count_{method}_gop{gop}_threshold_{threshold}.csv"
            graph_pdf_path = os.path.join(graph_folder, graph_pdf_name)
            plt.savefig(graph_pdf_path, format='png')
            plt.close()

            print(f"[INFO] 분석 완료 - {filename}")
            print(f"       CSV 저장: {result_csv_path}")
            print(f"       PDF 저장: {graph_pdf_path}")


[INFO] 분석 완료 - raw_data_kt_gop1_threshold_30.csv
       CSV 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/freeze_count_kt_gop1_threshold_30.csv
       PDF 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/freeze_count_kt_gop1_threshold_30.csv
[INFO] 분석 완료 - raw_data_kt_gop1_threshold_35.csv
       CSV 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/freeze_count_kt_gop1_threshold_35.csv
       PDF 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/freeze_count_kt_gop1_threshold_35.csv
[INFO] 분석 완료 - raw_data_kt_gop1_threshold_40.csv
       CSV 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/freeze_count_kt_gop1_threshold_40.csv
       PDF 저장: /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/freeze_count_kt_gop1_threshold_40.csv
[INFO] 분석 완료 - raw_data_kt_gop10_thre

In [111]:
import pandas as pd
import os
import re

# method, gop, threshold에 따른 파일명 생성
methods = ["kt", "lg", "combine"]
gops = [1, 10, 30]
thresholds = [30, 35, 40]

# CSV 파일 목록 생성
csv_files = [
    f"raw_data_{method}_gop{gop}_threshold_{threshold}.csv"
    for method in methods
    for gop in gops
    for threshold in thresholds
]

# 로그 저장할 리스트 초기화
log_data = []

# CSV 파일 경로 설정
base_path = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/"

# 각 파일에 대해 필터링 및 로그 작성
for csv_file in csv_files:
    file_path = os.path.join(base_path, csv_file)

    try:
        # CSV 파일 읽기
        df = pd.read_csv(file_path)

        # 파일명에서 method, gop, threshold 추출
        match = re.match(r"raw_data_(\w+)_gop(\d+)_threshold_(\d+)\.csv", csv_file)
        if match:
            method, gop, threshold = match.groups()
            gop = int(gop)
            threshold = int(threshold)
        else:
            raise ValueError(f"Filename format is incorrect: {csv_file}")

        # 필터링 조건 적용
        filtered_df = df[df['ssim'] == 0]
        filtered_count = len(filtered_df)
        
        # 추가 조건: network_latency_ms < threshold인 경우
        low_latency_filtered_df = df[(df['ssim'] == 0) & (df['network_latency_ms'] < threshold)]
        low_latency_filtered_count = len(low_latency_filtered_df)

        # 로그 데이터 추가
        log_data.append({
            "file_name": csv_file,
            "method": method,
            "gop": gop,
            "threshold": threshold,
            "filtered_count": filtered_count,
            "low_latency_filtered_count": low_latency_filtered_count
        })

        print(f"Processed {csv_file}: Filtered data count = {filtered_count}, Low latency filtered count = {low_latency_filtered_count}")
    except Exception as e:
        # 에러 발생 시 로그 추가
        log_data.append({
            "file_name": csv_file,
            "method": "unknown",
            "gop": "unknown",
            "threshold": "unknown",
            "error": str(e)
        })
        print(f"Error processing {csv_file}: {e}")

# 로그 데이터 저장
log_df = pd.DataFrame(log_data)
log_file_path = os.path.join(base_path, "filter_log.csv")
log_df.to_csv(log_file_path, index=False)

print(f"Log saved to {log_file_path}")


Processed raw_data_kt_gop1_threshold_30.csv: Filtered data count = 4476, Low latency filtered count = 0
Processed raw_data_kt_gop1_threshold_35.csv: Filtered data count = 1577, Low latency filtered count = 0
Processed raw_data_kt_gop1_threshold_40.csv: Filtered data count = 655, Low latency filtered count = 0
Processed raw_data_kt_gop10_threshold_30.csv: Filtered data count = 6856, Low latency filtered count = 2379
Processed raw_data_kt_gop10_threshold_35.csv: Filtered data count = 2518, Low latency filtered count = 941
Processed raw_data_kt_gop10_threshold_40.csv: Filtered data count = 989, Low latency filtered count = 334
Processed raw_data_kt_gop30_threshold_30.csv: Filtered data count = 7079, Low latency filtered count = 2602
Processed raw_data_kt_gop30_threshold_35.csv: Filtered data count = 2564, Low latency filtered count = 987
Processed raw_data_kt_gop30_threshold_40.csv: Filtered data count = 1014, Low latency filtered count = 359
Processed raw_data_lg_gop1_threshold_30.csv: F

In [125]:
import pandas as pd
import os

folder_path = '/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/'
file_name = 'step_logs_1_1.csv'
file_path = os.path.join(folder_path, file_name)

# CSV 파일 읽기
df = pd.read_csv(file_path)

# SSIM이 0인 데이터 필터링
filtered_df = df[df['SSIM'] == 0]

# Frame loss가 True이고 SSIM이 0인 데이터 필터링
filtered_frame_loss_df = df[(df['SSIM'] == 0) & (df['Frame loss'] == True)]

# 필터링된 데이터 개수 출력
print("Filtered data count (SSIM == 0):", len(filtered_df))
print("Filtered data count (SSIM == 0 and Frame loss == True):", len(filtered_frame_loss_df))

# "Data Size"와 "Frame loss"의 전체 평균 계산
average_data_size = df['Data Size'].mean()
average_frame_loss = df['Frame loss'].astype(int).mean()  # True를 1, False를 0으로 변환 후 평균 계산

# 결과 출력
print(f"Average Data Size: {average_data_size}")
print(f"Average Frame Loss Rate: {average_frame_loss}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/step_logs_1_1.csv'

In [109]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def count_consecutive_zeros(ssim_values):
    """
    ssim_values(list or Series)에서 값이 0인 구간이 연속적으로 몇 개씩 나타나는지 카운트하여
    {연속개수: 빈도} 딕셔너리를 반환
    """
    consecutive_counts = {}
    current_count = 0
    
    for val in ssim_values:
        if val == 0:
            current_count += 1
        else:
            # 0이 아닌 값이 나타나면, 지금까지 센 연속 0의 길이를 기록
            if current_count > 0:
                consecutive_counts[current_count] = consecutive_counts.get(current_count, 0) + 1
                current_count = 0
    # 마지막 값이 0으로 끝났다면 처리
    if current_count > 0:
        consecutive_counts[current_count] = consecutive_counts.get(current_count, 0) + 1
    
    return consecutive_counts

ssim_values = df['SSIM'].fillna(0)  # 혹시 모를 NaN 방지
zero_counts = count_consecutive_zeros(ssim_values)

result_df = pd.DataFrame({
    'FreezeDuration': list(zero_counts.keys()),
    'Count': list(zero_counts.values())
}).sort_values(by='FreezeDuration', ascending=True)

# 결과 CSV 저장
result_csv_name = f"freeze_duration_40.csv"
result_csv_path = os.path.join(folder_path, result_csv_name)
result_df.to_csv(result_csv_path, index=False)


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------
# 사용자 환경에 맞게 경로 수정
# ------------------------------------------------

folder_path = '/home/songmu/multipath/rl/results/2025_02_21_20_19/evaluation/logs/step/'
FREEZE_DURATION_FOLDER = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration"
GRAPH_FOLDER = os.path.join(FREEZE_DURATION_FOLDER, "graph")

# 비교할 threshold 값
THRESHOLDS = [30, 35, 40]

# 분석 대상 (예시)
methods = ["kt", "lg", "combine"]
# methods = ["combine"]
gops = [1, 10, 30]

# 참조 CSV 파일 (FreezeDuration, Count)
PDF_REFERENCE_FILES = {
    30: "freeze_duration_30.csv",
    35: "freeze_duration_35.csv",
    40: "freeze_duration_40.csv"
}

# 마커 크기와 선 두께
MARKER_SIZE = 0.5
LINE_WIDTH = 1

def load_and_compute_pdf_cdf(threshold):
    """
    주어진 threshold에 대해,
    1) freeze_count_{method}_gop{gop}_threshold_{threshold}.csv 파일 (여러 개)
    2) pdf_{threshold}.csv (1개, 참조용)
    
    각각 (x_values, pdf_values, cdf_values, label)을 담은 리스트를 반환.
    """
    result_lines = []  # (x_values, pdf_values, cdf_values, label)
    
    # ---------- (1) freeze_count_... ----------
    for method in methods:
        for gop in gops:
            csv_name = f"freeze_count_{method}_gop{gop}_threshold_{threshold}.csv"
            csv_path = os.path.join(FREEZE_DURATION_FOLDER, csv_name)
            
            if not os.path.exists(csv_path):
                print(f"[WARNING] Not found: {csv_path}")
                continue
            
            df = pd.read_csv(csv_path)
            if 'FreezeDuration' not in df.columns or 'Count' not in df.columns:
                print(f"[WARNING] Invalid CSV format: {csv_path}")
                continue

            # PDF, CDF 계산
            total_count = df['Count'].sum()
            if total_count > 0:
                df['PDF'] = df['Count'] / total_count
            else:
                df['PDF'] = 0

            # FreezeDuration 기준 정렬 후 CDF 계산
            df = df.sort_values(by='FreezeDuration').reset_index(drop=True)
            df['CDF'] = df['PDF'].cumsum()

            label_name = f"{method}_gop{gop}"
            
            # x, pdf, cdf, label 저장
            x_values = df['FreezeDuration'].values
            pdf_values = df['PDF'].values
            cdf_values = df['CDF'].values
            
            result_lines.append((x_values, pdf_values, cdf_values, label_name))
    
    # ---------- (2) pdf_{threshold}.csv (참조 파일) ----------
    ref_csv_name = PDF_REFERENCE_FILES.get(threshold, None)
    if ref_csv_name:
        ref_csv_path = os.path.join(folder_path, ref_csv_name)
        if os.path.exists(ref_csv_path):
            df_ref = pd.read_csv(ref_csv_path)
            if 'FreezeDuration' not in df_ref.columns or 'Count' not in df_ref.columns:
                print(f"[WARNING] Invalid reference CSV format: {ref_csv_path}")
            else:
                total_count_ref = df_ref['Count'].sum()
                if total_count_ref > 0:
                    df_ref['PDF'] = df_ref['Count'] / total_count_ref
                else:
                    df_ref['PDF'] = 0

                df_ref = df_ref.sort_values(by='FreezeDuration').reset_index(drop=True)
                df_ref['CDF'] = df_ref['PDF'].cumsum()
                
                # -------------------------------------------
                # 범례(legend) 이름: pdf_30 → RL_30, pdf_35 → RL_35
                # -------------------------------------------
                if threshold == 30:
                    ref_label = "RL_30"
                elif threshold == 35:
                    ref_label = "RL_35"
                else:
                    ref_label = f"RL_{threshold}"  # 필요 시 확장 가능
                
                x_values = df_ref['FreezeDuration'].values
                pdf_values = df_ref['PDF'].values
                cdf_values = df_ref['CDF'].values

                result_lines.append((x_values, pdf_values, cdf_values, ref_label))
        else:
            print(f"[WARNING] Reference file not found: {ref_csv_path}")
    else:
        print(f"[WARNING] No reference file specified for threshold={threshold}")

    return result_lines

def plot_pdf_only(threshold, lines):
    """
    lines: (x_values, pdf_values, cdf_values, label) 형태 리스트
    PDF만 그려서 PNG로 저장
    """
    plt.figure(figsize=(8,5))
    for (x, pdf, cdf, label) in lines:
        plt.plot(
            x, pdf,
            marker='o', linestyle='-',
            markersize=MARKER_SIZE, linewidth=LINE_WIDTH,
            label=f"{label} (PDF)"
        )
    
    plt.title(f"Comparison of Freeze Duration - PDF (threshold={threshold})")
    plt.xlabel("Freeze Duration (Consecutive 0 count)")
    plt.ylabel("PDF")
    plt.grid(True)
    plt.legend()
    plt.xlim([0, 50])  # 필요 시 수정

    # -------------------------------
    # PNG로 저장 (기존 pdf 대신)
    # -------------------------------
    out_png_name = f"compare_threshold_{threshold}_pdf.png"
    out_png_path = os.path.join(GRAPH_FOLDER, out_png_name)
    plt.savefig(out_png_path, format='png', dpi=300)
    plt.close()
    print(f"[INFO] Saved PDF-only figure -> {out_png_path}")

def plot_cdf_only(threshold, lines):
    """
    lines: (x_values, pdf_values, cdf_values, label) 형태 리스트
    CDF만 그려서 PNG로 저장하고, raw data를 CSV 파일로 저장.
    """

    all_x_values = []
    all_cdf_values = []

    plt.figure(figsize=(8,5))

    # CDF 데이터 저장을 위한 리스트
    cdf_data_list = []

    for (x, pdf, cdf, label) in lines:
        plt.plot(
            x, cdf,
            marker='o', linestyle='--',
            markersize=MARKER_SIZE, linewidth=LINE_WIDTH,
            label=f"{label} (CDF)"
        )

        all_x_values.extend(x)
        all_cdf_values.extend(cdf)

        # CDF 데이터 저장 (FreezeDuration, CDF, Label)
        for i in range(len(x)):
            cdf_data_list.append([x[i], cdf[i], label])

    # x, y축 최소/최대값 자동 조정
    min_x_value = max(0, min(all_x_values)) if all_x_values else 0
    min_cdf_value = max(0.01, min(all_cdf_values)) if all_cdf_values else 0
    
    plt.title(f"Comparison of Freeze Duration - CDF (threshold={threshold})")
    plt.xlabel("Freeze Duration")
    plt.ylabel("CDF")
    plt.grid(True)
    plt.legend()
    plt.xlim([min_x_value, 50])
    plt.ylim([min_cdf_value, 1])

    # -------------------------------
    # PNG로 저장 (기존 pdf 대신)
    # -------------------------------
    out_png_name = f"compare_threshold_{threshold}_cdf.png"
    out_png_path = os.path.join(GRAPH_FOLDER, out_png_name)
    plt.savefig(out_png_path, format='png', dpi=300)
    plt.close()
    print(f"[INFO] Saved CDF-only figure -> {out_png_path}")

    # -------------------------------
    # CSV로 저장 (CDF raw data)
    # -------------------------------
    cdf_data_folder = os.path.join(GRAPH_FOLDER, "cdf_data")
    os.makedirs(cdf_data_folder, exist_ok=True)  # 폴더 생성

    out_csv_name = f"cdf_data_threshold_{threshold}.csv"
    out_csv_path = os.path.join(cdf_data_folder, out_csv_name)

    df_cdf = pd.DataFrame(cdf_data_list, columns=["FreezeDuration", "CDF", "Label"])
    df_cdf.to_csv(out_csv_path, index=False)

    print(f"[INFO] Saved CDF raw data -> {out_csv_path}")
if __name__ == "__main__":
    os.makedirs(GRAPH_FOLDER, exist_ok=True)

    for thr in THRESHOLDS:
        # (1) 데이터 로드 & PDF/CDF 계산
        lines_data = load_and_compute_pdf_cdf(thr)

        # (2) PDF 전용 그래프 그리기
        # plot_pdf_only(thr, lines_data)

        # (3) CDF 전용 그래프 그리기
        plot_cdf_only(thr, lines_data)


[INFO] Saved CDF-only figure -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/compare_threshold_30_cdf.png
[INFO] Saved CDF raw data -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/cdf_data/cdf_data_threshold_30.csv
[INFO] Saved CDF-only figure -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/compare_threshold_35_cdf.png
[INFO] Saved CDF raw data -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/cdf_data/cdf_data_threshold_35.csv
[INFO] Saved CDF-only figure -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/compare_threshold_40_cdf.png
[INFO] Saved CDF raw data -> /home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/freeze_duration/graph/cdf_data/cdf_data_threshold_40.csv


In [104]:
import pandas as pd

# CSV 파일 경로 설정
file_path = "/home/songmu/multipath/server/logs/2025_02_17_13_57/baseline/60/raw_data_combine_gop30_threshold_40.csv"  # 파일명을 실제 파일명으로 변경하세요.

# CSV 파일 로드
df = pd.read_csv(file_path)

# 1. network_latency_ms < 40 인 개수 및 전체 중 해당 부분을 뺀 비율 (prr)
latency_threshold = 40
count_below_threshold = (df["network_latency_ms"] > latency_threshold).sum()
prr = 1 - (count_below_threshold / len(df))

# 2. ssim = 0 인 개수
count_ssim_zero = (df["ssim"] == 0).sum()

# 3. datasize의 평균
datasize_mean = df["datasize"].mean()

# 4. network_latency_ms < 40 이면서 ssim = 0 인 개수
count_below_threshold_and_ssim_zero = ((df["network_latency_ms"] < latency_threshold) & (df["ssim"] == 0)).sum()

# 결과 출력
print(f"1. network_latency_ms < {latency_threshold} 개수: {count_below_threshold}, PRR: {prr:.4f}")
print(f"2. ssim = 0 인 개수: {count_ssim_zero}")
print(f"3. datasize 평균: {datasize_mean:.2f}")
print(f"4. network_latency_ms < {latency_threshold} 이면서 ssim = 0 인 개수: {count_below_threshold_and_ssim_zero}")



1. network_latency_ms < 40 개수: 124, PRR: 0.9926
2. ssim = 0 인 개수: 202
3. datasize 평균: 15670.74
4. network_latency_ms < 40 이면서 ssim = 0 인 개수: 78
